## 绘制人口迁徙平均指数

In [ ]:
#这一步都是转换数据的操作
#将dta转换成excel文件

import pandas as pd

# 加载 .dta 文件
dta_file_path = r"D:\A研究数据\人口迁徙数据\长三角年度城市间迁徙数据18-23.dta" 
data = pd.read_stata(dta_file_path)

# 将其保存为 Excel 文件
excel_file_path = r"D:\A研究数据\人口迁徙数据\长三角年度城市间迁徙数据18-23.xlsx"
data.to_excel(excel_file_path, index=False)

print(f"文件已转换并保存为: {excel_file_path}")

#计算城市对平均迁徙指数

# 加载Excel文件
df = pd.read_excel("/Users/zhaozijing/Desktop/ACSP/regression result/citypair_sep.9.xlsx")

# 创建城市对的标识，使得 "X-> Y" 和 "Y -> X" 被视为相同的城市对
df['city_pair'] = df.apply(lambda row: tuple(sorted([row['startE'], row['endE']])), axis=1)

# 计算每对城市之间的平均年度迁徙指数
df_avg = df.groupby(['city_pair', '年份'], as_index=False)['年度实际迁徙指数'].mean()

# 重新拆分城市对
df_avg['startE'] = df_avg['city_pair'].apply(lambda x: x[0])
df_avg['endE'] = df_avg['city_pair'].apply(lambda x: x[1])

# 准备导出数据的DataFrame
df_avg_export = df_avg[['city_pair', 'startE', 'endE', '年份', '年度实际迁徙指数']]
df_avg_export.columns = ['city_pair', 'cityi', 'cityj', 'year', '平均迁徙指数']

# 保存到Excel
output_file_path = "D:\\A研究数据\\人口迁徙数据\\POP平均迁徙指数.xlsx"
df_avg_export.to_excel(output_file_path, index=False)

print(f"文件已保存到 {output_file_path}")

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
from shapely.geometry import LineString
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import os


# 加载Excel文件
df = pd.read_excel("/Users/zhaozijing/Desktop/ACSP/regression result/citypair_sep.9.xlsx")

# 仅保留始发城市，终点城市，年份，迁徙平均指标
df = df[['city1', 'city2', 'year', 'avg_migration_index']]

# Define a standard mapping for commonly accepted English translations of Chinese city names
city_name_mapping = {
    "上海": "Shanghai",
    "杭州": "Hangzhou",
    "宁波": "Ningbo",
    "温州": "Wenzhou",
    "嘉兴": "Jiaxing",
    "湖州": "Huzhou",
    "绍兴": "Shaoxing",
    "金华": "Jinhua",
    "台州": "TAIzhou",
    "舟山": "Zhoushan",
    "衢州": "Quzhou",
    "丽水": "Lishui",
    "南京": "Nanjing",
    "无锡": "Wuxi",
    "常州": "Changzhou",
    "苏州": "SUzhou",
    "南通": "Nantong",
    "扬州": "Yangzhou",
    "镇江": "Zhenjiang",
    "泰州": "Taizhou",
    "徐州": "Xuzhou",
    "盐城": "Yancheng",
    "淮安": "Huaian",
    "连云港": "Lianyungang",
    "宿迁": "Suqian",
    "芜湖": "Wuhu",
    "马鞍山": "Maanshan",
    "铜陵": "Tongling",
    "安庆": "Anqing",
    "黄山": "Huangshan",
    "滁州": "Chuzhou",
    "阜阳": "Fuyang",
    "宿州": "Suzhou",
    "六安": "Luan",
    "池州": "Chizhou",
    "宣城": "Xuancheng",
    "合肥": "Hefei",
    "淮北": "Huaibei",
    "蚌埠": "Bengbu",
    "亳州": "Bozhou",
    "滁州": "Chuzhou",
    "淮南": "Huainan",
}

# Apply the mapping to city1 and city2 columns
df['city1'] = df['city1'].map(city_name_mapping)
df['city2'] = df['city2'].map(city_name_mapping)


# 避免多线程导致的内存泄漏问题
os.environ['OMP_NUM_THREADS'] = '1'
plt.rcParams['font.family'] = 'Times New Roman'

# 加载行政边界数据
city_boundaries = gpd.read_file("/Users/zhaozijing/Desktop/ACSP/长三角城市数据/长三角地市_1.shp").to_crs(epsg=3857)
# province_boundaries = gpd.read_file("/Users/zhaozijing/Desktop/ACSP/长三角城市数据/sheng2022.shp").to_crs(epsg=3857)

# 创建城市中心点字典
city_boundaries['centroid'] = city_boundaries['geometry'].centroid
city_centers = city_boundaries.set_index('cityE')['centroid'].to_dict()

# 使用城市中心点创建线段
def create_linestring(row):
    point1 = city_centers[row['city1']]
    point2 = city_centers[row['city2']]
    return LineString([point1, point2])

# 根据平均值生成线段
df['geometry'] = df.apply(create_linestring, axis=1)

# 转换为 GeoDataFrame
avg_migration_network = gpd.GeoDataFrame(df, geometry='geometry', crs=city_boundaries.crs)

# Define color mapping for imbalance index ranges
color_map = {
    '0-300': 'lightgrey',
    '300-1000': 'lightgreen',
    '1000-2500': 'orange',
    '2500-9000': 'red',
}

width_map = {
    '0-300': 0.3,
    '300-1000': 1.5,
    '1000-2500': 2.0,
    '2500-9000': 2.5,
}

# Categorize imbalance values
def get_color_and_width(value):
    if 0 <= value < 300:
        return color_map['0-300'], width_map['0-300']
    elif 300 <= value < 1000:
        return color_map['300-1000'], width_map['300-1000']
    elif 1000 <= value < 2500:
        return color_map['1000-2500'], width_map['1000-2500']
    elif 2500 <= value < 9000:
        return color_map['2500-9000'], width_map['2500-9000']
    return 'grey', 0.5  # Default color for any value outside specified ranges

avg_migration_network['color'], avg_migration_network['width'] = zip(*avg_migration_network['avg_migration_index'].apply(get_color_and_width))


# 绘制地图
for year in avg_migration_network['year'].unique():
    fig, ax = plt.subplots(1, figsize=(10, 12))
    # province_boundaries.plot(ax=ax, color='none', edgecolor='black')
    city_boundaries.plot(ax=ax, color='none', edgecolor='black')

    # 添加城市名称
    for idx, row in city_boundaries.iterrows():
        ax.text(row.geometry.centroid.x, row.geometry.centroid.y, row['cityE'], fontsize=9, ha='center', va='center', color='black', zorder=6)
        ax.plot(row.geometry.centroid.x, row.geometry.centroid.y, 'o', color='red', markersize=5, zorder=4)  # 添加红色中心点

  # 绘制平均迁徙指数的物流线段
    year_data = avg_migration_network[avg_migration_network['year'] == year]
    for _, row in year_data.iterrows():
        line = row['geometry']
        color = row['color']
        width = row['width']
        ax.plot(*line.xy, color=color, linewidth=width)
    # year_data.plot(ax=ax, linewidth=year_data['migration_imbalance'], color=year_data['color'], zorder=2)
    
    # Define legend elements for line thickness and color based on migration imbalance index
    line_elements = [
    mlines.Line2D([], [], color='lightgrey', linewidth=0.3, label='0-300'),
    mlines.Line2D([], [], color='lightgreen', linewidth=1.5, label='300-1000'),
    mlines.Line2D([], [], color='orange', linewidth=2.0, label='1000-2500'),
    mlines.Line2D([], [], color='red', linewidth=2.5, label='2500-9000')]
    
    legend1 = ax.legend(handles=line_elements, loc='lower left', bbox_to_anchor=(0.05, 0.1),fontsize=10, title="Average Migration Index")

    # 手动绘制比例尺
    scalebar_location = (0.11, 0.06)  # 调整比例尺位置
    scale_length = 100000  # 100 km的比例尺
    x_length = scale_length / (ax.get_xlim()[1] - ax.get_xlim()[0])

    # 绘制比例尺线
    ax.plot([scalebar_location[0], scalebar_location[0] + x_length],
            [scalebar_location[1], scalebar_location[1]],
            color='black', transform=ax.transAxes, lw=2)

    # 添加比例尺文本
    ax.text(scalebar_location[0] + x_length / 2, scalebar_location[1] - 0.01, '100 km',
            horizontalalignment='center', verticalalignment='top',
            transform=ax.transAxes, fontsize=12)

    # 添加北箭头
    north_arrow_img = plt.imread("/Users/zhaozijing/Downloads/north_arrow1.png")
    imagebox = OffsetImage(north_arrow_img, zoom=0.5)
    ab = AnnotationBbox(imagebox, (0.95, 0.95), xycoords='axes fraction', frameon=False)
    ax.add_artist(ab)

    # 设置年份标题和去除坐标轴
    ax.set_title(f'Year {year}', loc='center')
    ax.set_xticks([])
    ax.set_yticks([])

    # 保存图片并显示
    plt.title(f"Average Migration Index {year}", fontdict={'fontname': 'Times New Roman', 'fontsize': 15})
    plt.axis('off')
    plt.savefig(f"/Users/zhaozijing/Downloads/图/avg_migration_{year}.jpg", format='jpg', dpi=300)
    plt.show()

## 绘制合作网络图 （需要按类别处理一下，图已经有了）

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import LineString
from matplotlib.patches import Patch, PathPatch, Circle,Polygon
from matplotlib.collections import PatchCollection
from matplotlib.path import Path
from matplotlib.lines import Line2D
import os
import numpy as np
# 设置环境变量以避免多线程导致的内存泄漏问题
os.environ['OMP_NUM_THREADS'] = '1'

# 加载行政边界数据
city_boundaries = gpd.read_file("D:\\A研究数据\\地图数据\\2022年地市边界\\长三角地市_1.shp").to_crs(epsg=3857)
#province_boundaries = gpd.read_file("D:\\A研究数据\\地图数据\\2022年地市边界\\长三角省级边界.shp").to_crs(epsg=3857)

# 加载Excel文件
df = pd.read_excel("D:\\A研究数据\\区域合作数据\\城市对_年份.xlsx")
# 计算外接圆中心和半径的函数
def get_circle(x1, y1, x2, y2, x3, y3):
    a = x1 - x2
    b = y1 - y2
    c = x1 - x3
    d = y1 - y3
    a1 = ((x1 ** 2 - x2 ** 2) + (y1 ** 2 - y2 ** 2)) / 2.0
    a2 = ((x1 ** 2 - x3 ** 2) + (y1 ** 2 - y3 ** 2)) / 2.0
    theta = b * c - a * d
    if abs(theta) < 1e-7:
        raise ValueError("Three collinear points do not define a circle.")
    x0 = (b * a2 - d * a1) / theta
    y0 = (c * a1 - a * a2) / theta
    r = np.sqrt((x1 - x0) ** 2 + (y1 - y0) ** 2)
    return x0, y0, r

# 添加指北针的函数
def add_north(ax, x, y, text_size, arrow_width, text_pad, arrow_height, line_width, add_circle):
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    width = x_max - x_min
    height = y_max - y_min

    left = (x_min + width * (x - arrow_width / 2), y_min + height * (y - arrow_height))
    right = (x_min + width * (x + arrow_width / 2), left[1])
    top = (x_min + width * x, y_min + height * y)

    left_patch = Polygon([left, top, (top[0], left[1])], closed=True, edgecolor='black', facecolor='black')
    ax.add_patch(left_patch)

    right_patch = Polygon([right, top, (top[0], left[1])], closed=True, edgecolor='black', facecolor='white')
    ax.add_patch(right_patch)

    if add_circle:
        x0, y0, r = get_circle(left[0], left[1], top[0], top[1], right[0], right[1])
        circle = Circle((x0, y0), r, edgecolor='black', facecolor='none', linewidth=line_width)
        ax.add_patch(circle)
        ax.text(x0, y0 + 1.2*r + text_pad, 'N', horizontalalignment='center', verticalalignment='center', 
                fontsize=text_size, family='Arial', zorder=5)
# 获取年份列表
years = df['year'].unique().tolist()
# 创建核心城市和非核心城市的分类
city_names = city_boundaries[['cityE', 'geometry']]
core_cities = ['Bengbu', 'Changzhou', 'Chuzhou', 'Hangzhou', 'Hefei', 'Huaian', 'Huainan', 'Huangshan', 'Huzhou', 'Jiaxing', 'Liuan', 'Maanshan', 'Nanjing', 'Nantong', 'Ningbo', 'Quzhou', 'Shanghai', 'Shaoxing', 'SUzhou', 'TAIzhou', 'Wuhu', 'Wuxi', 'Xuancheng', 'Yangzhou', 'Zhenjiang', 'Zhoushan']
city_names['city_type'] = 'peripheral'
city_names.loc[city_names['cityE'].isin(core_cities), 'city_type'] = 'core'

# 定义颜色映射
color_map = {'core': 'white', 'peripheral': '#f5f5f5'}

# 创建城市中心点字典
city_centers = city_boundaries.set_index('cityE')['geometry'].centroid.to_dict()

# 使用城市中心点创建线段
def create_linestring(row):
    point1 = city_centers[row['cityiE']]
    point2 = city_centers[row['cityjE']]
    return LineString([point1, point2])

df['geometry'] = df.apply(create_linestring, axis=1)
logistics_network = gpd.GeoDataFrame(df, geometry='geometry', crs=city_boundaries.crs)

# 绘制地图
for year in df['year'].unique():
    fig, ax = plt.subplots(1, figsize=(10, 12))
    province_boundaries.plot(ax=ax, color='none', edgecolor='grey')
    city_boundaries.plot(ax=ax, color='none', edgecolor='grey')
     # 添加城市名称，并根据城市类型设置填充颜色
    for idx, row in city_names.iterrows():
        city_boundary = city_boundaries[city_boundaries['cityE'] == row['cityE']]
        city_boundary.plot(ax=ax, color=color_map[row['city_type']], edgecolor='grey')
        ax.text(row.geometry.centroid.x, row.geometry.centroid.y, row['cityE'], fontsize=8, ha='center', va='center', color='black')
    # 绘制物流线段
    year_data = logistics_network[logistics_network['year'] == year]
    year_data.plot(ax=ax, linewidth=year_data['数据'] / 100, color='#AFC2DC', zorder=2)

    # 标注指定城市中心点红圈，并设置渐变色
    cities_to_mark = ['Nanjing', 'Hefei', 'Shanghai', 'Hangzhou']
    patches = []
    for city in cities_to_mark:
        city_point = city_centers[city]
        circle = Circle((city_point.x, city_point.y), 5000, color='red', alpha=0.5)  # 可以调整alpha来实现颜色的渐变效果
        patches.append(circle)
    
    p = PatchCollection(patches, match_original=True, zorder=3)
    ax.add_collection(p)

     # 手动绘制比例尺
    scalebar_location = (0.11, 0.06)  # 更进一步下移比例尺，确保不遮挡地图
    scale_length = 100000 
    x_length = scale_length / (ax.get_xlim()[1] - ax.get_xlim()[0])

    # 绘制比例尺线
    ax.plot([scalebar_location[0], scalebar_location[0] + x_length],
            [scalebar_location[1], scalebar_location[1]],
            color='black', transform=ax.transAxes, lw=2)

    # 添加比例尺文本
    ax.text(scalebar_location[0] + x_length / 2, scalebar_location[1] - 0.01, '100 km',
            horizontalalignment='center', verticalalignment='top',
            transform=ax.transAxes, fontsize=12)
    # # 设置正确的比例和界限
    # ax.set_aspect('equal')
    
    # 添加指北针到你的地图
    add_north(ax, x=0.88, y=0.95, text_size=10, arrow_width=0.02, 
              text_pad=0.01, arrow_height=0.1, line_width=0.5, add_circle=True)
    # 添加自定义图例
    legend_elements = [
    Patch(facecolor='#f5f5f5', edgecolor='lightgrey', label='Periphery'),
    Patch(facecolor='none', edgecolor='k', label='Core'),
    Line2D([0], [0], color='#AFC2DC', lw=4, label='Cooperation Frequency'),
    Line2D([0], [0], marker='o', color='w', markersize=10, markerfacecolor='red', label='Provincial Capitals')
]


    leg = ax.legend(handles=legend_elements, title='LEGEND', loc='upper left', bbox_to_anchor=(0.092,0.22), fontsize='small', title_fontsize='medium')
    leg.get_frame().set_linewidth(0.0)
    # 添加年份标题
    ax.set_title(f'Year {year}', loc='center')
    # 去除横纵轴的数值
    ax.set_xticks([])
    ax.set_yticks([])
    # 设置图标题和其他参数
    plt.title(f"COOP {year}", fontdict={'fontname': 'Times New Roman', 'fontsize': 15})
    plt.axis('off')
    plt.savefig(f"D:\\A研究数据\\政府工作报告\\reg\\地图\\网络图plus\\network_{year}.jpg", format='jpg',dpi=300)
    plt.show()
